In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("data_1.csv")
data = data.sort_values(by=['messdaten'])
tolerance = 0.028

## Funktion zum gruppieren der Messdaten

In [3]:
def get_bins_b2t(data, top=4, corridor=(1.75, 1.8)):
    ''' Diese Funktion fässt jeweis 5 Messungen so zusammen,
        dass der jeweilige Korridor und die max_dif eingehalten werden.
        
        Der Ansatz besteht darin, immer kleinere Tabellen aus 5 Daten zu
        erstellen die den Anforderungen gerecht werden und daraus eine
        List mit Tbellen zu erstllen. Die Ursprungsliste wird nach und nach 
        abgebaut
    '''
    
    df = data
    bin_dfs=[]
    bin_neu = []
    tops = 0
    
    while len(df.messdaten)>0:  # solange noch mehr als 0 Messdaten vorhanden sind 
            if bin_neu ==[]:
                bin_neu = df[0:1] # nimm den kleinsten Wert der Tabelle
                df = df[df["serial#"] != bin_neu.iloc[-1]["serial#"]]#Beseitige diese Reihe aus der Ausgangstabelle
                
            # mit dem 1. niedrigen Wert wird nun geguckt, wie hoch die anderen 4 Werte durchschnittlich sein müssten, 
            # sodass die aufsummierte 5er-Reihe mindestens die untere Korridorschwelle schafft.
            # mit dieser Information wird dann eine subtabelle erstellt ausschließlich mit Werten darüber
            # Diesen Vorgan wiederholt er bis er 5 Elemente voll hat.
            
            for i in range(4):
                ziel_avg = (corridor[0]-np.sum(bin_neu.messdaten))/(5-len(bin_neu.messdaten))
                bin_neu = pd.concat([bin_neu, df[df.messdaten > ziel_avg].head(1)], axis=0)
                df = df[df["serial#"] != bin_neu.iloc[-1]["serial#"]]
            bin_dfs.append(bin_neu)
            bin_neu=[]
                
            # Da es mit diesem Algorithmus vorkommen kann, dass hiermit die obere Schwelle gerissen
            # werden kann, wird abwechselnd das gleiche von der oberen Schwelle ausgehend gemacht
            # "tops" ist die anzahl der mindestwechsel. tops kann in der Menge iteriert werden
            
            if tops < top:
                if bin_neu ==[]:
                    bin_neu = df.tail(1)
                    df = df[df["serial#"] != bin_neu.iloc[-1]["serial#"]]
                for i in range(4):
                    ziel_avg = (corridor[1]-np.sum(bin_neu.messdaten))/(5-len(bin_neu.messdaten))
                    bin_neu = pd.concat([bin_neu, df[df.messdaten < ziel_avg].tail(1)], axis=0)
                    df = df[df["serial#"] != bin_neu.iloc[-1]["serial#"]]
                bin_dfs.append(bin_neu)
                bin_neu=[]
                tops+=1


    return bin_dfs

In [4]:
def create_bin_stats(X):
    #set bin ids
    bin_no = []
    no = 1
    for i in range(len(X)//5):
        for j in range(5):
            bin_no.append(no)
        no+=1
    X["bin"] = bin_no
    bin_stats = X.groupby(by="bin").agg({"messdaten":"sum"}).rename(columns = {"messdaten":"bin_sum"})
    bin_stats["bin_dif"] = X.groupby(by="bin").agg({"messdaten":"max"}) - X.groupby(by="bin").agg({"messdaten":"min"})
    X = X.merge(bin_stats, on="bin")
    return X

In [5]:
def create_new_list(bin_list):
    ''' diese Funktion nimmt eine Liste aus listen und verkettet sie vertikal
        damit diese vernünftig ausgegeben werden kann'''
    start = bin_list[0]
    for i in bin_list[1:]:
        start=pd.concat([start, i], axis=0)
    cleaned_list = create_bin_stats(start)
    return cleaned_list

In [6]:
def get_input_stats(data, tolerance=0.028):
    ''' Diese Funktion nimmt den EIngangsdatensatz und bestimmt den größten und kleinsten
        Wert, den Durchschnittswert und die 5% max_dif (tolerance 2.5%)'''
    lim_upper = data["messdaten"].max()
    lim_lower= data["messdaten"].min()
    mean_df = data["messdaten"].mean()
    thresh_5_percent = data["messdaten"].mean()*(2*tolerance)
    return lim_upper, lim_lower, mean_df, thresh_5_percent

In [7]:
print(get_input_stats(data))

(0.2461, 0.2125, 0.23037919999999998, 0.0129012352)


In [8]:
def iterate_narrowing(data, tolerance, increase=1, decrease=1):
    corridor =(round(get_input_stats(data, tolerance)[1]*5, 2), round(get_input_stats(data, tolerance)[0]*5, 2))
    max_dif = get_input_stats(data, tolerance)[3]
    Result = False
    Result_variables = []
    flip_switch = 1
    
    while corridor[0] < corridor[1]:
        print(f"Testing corridor {corridor[0]} and {corridor[1]}")
        for i in range(5):
            result = create_new_list(get_bins_b2t(data, top=i, corridor=corridor))
            if min(np.array(result.bin_sum)) >= corridor[0] and max(np.array(result.bin_sum)) <= corridor[1] and max(np.array(result.bin_dif)) <= max_dif:
                Result = result
                print(f'results saved')
            break
        if flip_switch ==1:
            flip_switch = 0
            corridor = (round(corridor[0]+0.005*increase, 3), round(corridor[1],3))
        else:
            flip_switch = 1
            corridor = (round(corridor[0],3), round(corridor[1]-0.005*decrease,3)) 
    
    corridor_span = max(np.array(Result.bin_sum)) - min(np.array(Result.bin_sum))
    return Result, corridor_span

In [9]:
def get_best_Result(data, tolerance, increments = [1, 2, 3, 4]):
    Best_Result = (pd.DataFrame(), 300)
    for inc in increments:
        results = iterate_narrowing(data, tolerance, increase=inc)
        if results[1] < Best_Result[1]:
            Best_Result = results
        results = iterate_narrowing(data, tolerance, decrease=inc)
        if results[1] < Best_Result[1]:
            Best_Result = results
    Best_Result[0].to_csv("Ergebnis.csv")
    print(f' Untere Bin-Grenze= {min(np.array(Best_Result[0].bin_sum))}')
    print(f' Obere Bin-Grenze= {max(np.array(Best_Result[0].bin_sum))}')
    print(f' Maximale Varianz= {max(np.array(Best_Result[0].bin_dif))}')
    return "Process completed" 

In [10]:
def iterate_narrowing_2(data, tolerance):
    corridor =(round(get_input_stats(data, tolerance)[1]*5, 2), round(get_input_stats(data, tolerance)[0]*5, 2))
    max_dif = get_input_stats(data, tolerance)[3]
    Result = False
    corridor_span = 1
    corridor_bottom = corridor[0]
    corridor_top = corridor[1]
    max_span = corridor_top - corridor_bottom
    bottom_ref = corridor_bottom+max_span*0.3
    top_ref = corridor_top-max_span*0.3
    corridor_ref = 5*round(get_input_stats(data, tolerance)[2], 3)
    increments = np.linspace(corridor_bottom, corridor_top, 1+int(100*(corridor_top-corridor_bottom)))
    no_planned = round((len(increments)*0.7)**2)
    no_start = 1
    
    for inc in increments:
        for inc2 in increments:
            # hier wird die obere grenze iteriert
            if inc <= top_ref and inc2 >= bottom_ref:
                print(f"{no_start}/{no_planned} - Testing: {inc} : {inc2}")
                no_start +=1
                for i in range(10):
                    result = create_new_list(get_bins_b2t(data, top=i, corridor=(inc, inc2)))
                    if min(np.array(result.bin_sum)) >= inc and max(np.array(result.bin_sum)) <= inc2 and max(np.array(result.bin_dif)) <= max_dif and inc2-inc <corridor_span:
                        corridor_span = inc2-inc
                        Result = result
                        print(f'results saved with max corridor span of {corridor_span}')
            # hier wird die untere grenze iteriert
            if inc >= bottom_ref and inc2 <=top_ref:
                print(f"{no_start}/{no_planned} - Testing: {inc2} : {inc}")
                no_start +=1
                for i in range(10):
                    result = create_new_list(get_bins_b2t(data, top=i, corridor=(inc2, inc)))
                    if min(np.array(result.bin_sum)) >= inc2 and max(np.array(result.bin_sum)) <= inc and max(np.array(result.bin_dif)) <= max_dif and inc-inc2 <corridor_span:
                        corridor_span = inc-inc2
                        Result = result
                        print(f'results saved with max corridor span of {corridor_span}')
    if Result:
        Result.to_csv("Ergebnis.csv")
        print(f' Untere Bin-Grenze= {min(np.array(Result.bin_sum))}')
        print(f' Obere Bin-Grenze= {max(np.array(Result.bin_sum))}')
        print(f' Maximale Varianz= {max(np.array(Result.bin_dif))}')
    else:
        print("There was an error")
    return "Process completed" 

In [12]:
def iterate_narrowing_3(data, tolerance):
    corridor =(round(get_input_stats(data, tolerance)[1]*5, 2), round(get_input_stats(data, tolerance)[0]*5, 2))
    max_dif = get_input_stats(data, tolerance)[3]
    Result = False
    corridor_span = 1
    corridor_bottom = corridor[0]
    corridor_top = corridor[1]
    max_span = corridor_top - corridor_bottom
    bottom_ref = corridor_bottom+max_span*0.3
    top_ref = corridor_top-max_span*0.3
    corridor_ref = 5*round(get_input_stats(data, tolerance)[2], 3)
    increments = np.linspace(corridor_bottom, corridor_top, 1+int(100*(corridor_top-corridor_bottom)))
    no_planned = 0
    no_start = 1

    
    ### Bestimmen der Anzahl der Iterationen ###
    
    for inc in increments:
        increments_top = [x for x in increments if x >inc]
        increments_bot = [x for x in increments if x < inc]
        for top in increments_top:
            no_planned +=1
        for bot in increments_bot:
            no_planned +=1
    
    ### Ab hier die echte Iteration ###
    
    for inc in increments:
        increments_top = [x for x in increments if x >inc]
        increments_bot = [x for x in increments if x < inc]
        for top in increments_top:
            print(f"{no_start}/{no_planned} - Testing: {inc} : {top}")
            no_start +=1
            if top - inc < corridor_span:
                for i in range(10):
                    result = create_new_list(get_bins_b2t(data, top=i, corridor=(inc, top)))
                    if min(np.array(result.bin_sum)) >= inc and max(np.array(result.bin_sum)) <= top and max(np.array(result.bin_dif)) <= max_dif and top-inc < corridor_span:
                        corridor_span = top-inc
                        Result = result
                        print(f'results stored max corridor = {corridor_span} from {inc} to {top}')
                        
        for bot in increments_bot:
            print(f"{no_start}/{no_planned} - Testing: {bot} : {inc}")
            no_start +=1
            if inc-bot < corridor_span:
                for i in range(10):
                    result = create_new_list(get_bins_b2t(data, top=i, corridor=(bot, inc)))
                    if min(np.array(result.bin_sum)) >= bot and max(np.array(result.bin_sum)) <= inc and max(np.array(result.bin_dif)) <= max_dif and inc-bot <corridor_span:
                        corridor_span = inc-bot
                        Result = result
                        print(f'results stored max corridor = {corridor_span} from {bot} to {inc}')
    Result.to_csv("Ergebnis.csv")
    print(f' Untere Bin-Grenze= {min(np.array(Result.bin_sum))}')
    print(f' Obere Bin-Grenze= {max(np.array(Result.bin_sum))}')
    print(f' Maximale Varianz= {max(np.array(Result.bin_dif))}')
   
    return print("Sorting completed")

In [13]:
iterate_narrowing_3(data, tolerance)

1/272 - Testing: 1.06 : 1.0706250000000002
2/272 - Testing: 1.06 : 1.08125
3/272 - Testing: 1.06 : 1.091875
4/272 - Testing: 1.06 : 1.1025
5/272 - Testing: 1.06 : 1.1131250000000001
6/272 - Testing: 1.06 : 1.12375
7/272 - Testing: 1.06 : 1.134375
8/272 - Testing: 1.06 : 1.145
9/272 - Testing: 1.06 : 1.1556250000000001
10/272 - Testing: 1.06 : 1.16625
11/272 - Testing: 1.06 : 1.176875
12/272 - Testing: 1.06 : 1.1875
13/272 - Testing: 1.06 : 1.198125
results stored max corridor = 0.13812500000000005 from 1.06 to 1.198125
14/272 - Testing: 1.06 : 1.20875
15/272 - Testing: 1.06 : 1.2193749999999999
16/272 - Testing: 1.06 : 1.23
17/272 - Testing: 1.0706250000000002 : 1.08125
18/272 - Testing: 1.0706250000000002 : 1.091875
19/272 - Testing: 1.0706250000000002 : 1.1025
20/272 - Testing: 1.0706250000000002 : 1.1131250000000001
21/272 - Testing: 1.0706250000000002 : 1.12375
22/272 - Testing: 1.0706250000000002 : 1.134375
23/272 - Testing: 1.0706250000000002 : 1.145
24/272 - Testing: 1.070625000

191/272 - Testing: 1.1556250000000001 : 1.176875
192/272 - Testing: 1.16625 : 1.176875
193/272 - Testing: 1.1875 : 1.198125
194/272 - Testing: 1.1875 : 1.20875
195/272 - Testing: 1.1875 : 1.2193749999999999
196/272 - Testing: 1.1875 : 1.23
197/272 - Testing: 1.06 : 1.1875
198/272 - Testing: 1.0706250000000002 : 1.1875
199/272 - Testing: 1.08125 : 1.1875
200/272 - Testing: 1.091875 : 1.1875
201/272 - Testing: 1.1025 : 1.1875
202/272 - Testing: 1.1131250000000001 : 1.1875
203/272 - Testing: 1.12375 : 1.1875
204/272 - Testing: 1.134375 : 1.1875
205/272 - Testing: 1.145 : 1.1875
206/272 - Testing: 1.1556250000000001 : 1.1875
207/272 - Testing: 1.16625 : 1.1875
208/272 - Testing: 1.176875 : 1.1875
209/272 - Testing: 1.198125 : 1.20875
210/272 - Testing: 1.198125 : 1.2193749999999999
211/272 - Testing: 1.198125 : 1.23
212/272 - Testing: 1.06 : 1.198125
213/272 - Testing: 1.0706250000000002 : 1.198125
214/272 - Testing: 1.08125 : 1.198125
215/272 - Testing: 1.091875 : 1.198125
216/272 - Testi

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [91]:
def iterate_narrowing_4(data, tolerance, steps = 10, span = 1):
    corridor =(round(get_input_stats(data, tolerance)[1]*5, 2), round(get_input_stats(data, tolerance)[0]*5, 2))
    max_dif = get_input_stats(data, tolerance)[3]
    Result = False
    corridor_span = span
    corridor_bottom = corridor[0]
    corridor_top = corridor[1]
    tolerance = tolerance*5
    max_span = corridor_top - corridor_bottom
    bottom_ref = corridor_bottom+max_span*0.3
    top_ref = corridor_top-max_span*0.3
    corridor_ref = 5*round(get_input_stats(data, tolerance)[2], 3)
    increments = [round(x, 3) for x in np.linspace(corridor_bottom, corridor_top, steps)]
    no_start = 1
    corridor_pairs = []
    
    ### Bestimmen der Anzahl der Iterationen ###
    
    for inc in increments:
        increments_top = [x for x in increments if x >inc]
        increments_bot = [x for x in increments if x < inc]
        for top in increments_top:
            pair = (inc, top)
            if pair not in corridor_pairs and top-inc < tolerance and top-inc < corridor_span:
                corridor_pairs.append(pair)
        for bot in increments_bot:
            pair = (bot, inc)
            if pair not in corridor_pairs and inc-bot < tolerance and inc-bot < corridor_span:
                corridor_pairs.append(pair)
    no_planned = len(corridor_pairs)
    
    ### Ab hier die echte Iteration ###
    
    for pair in corridor_pairs:
        print(f"{no_start}/{no_planned} - Testing: {pair}")
        no_start +=1
        if pair[1] - pair[0] < corridor_span:
            for i in range(10):
                result = create_new_list(get_bins_b2t(data, top=i, corridor=pair))
                if min(np.array(result.bin_sum)) >= pair[0] and max(np.array(result.bin_sum)) <= pair[1] and max(np.array(result.bin_dif)) <= max_dif:
                    corridor_span = max(np.array(result.bin_sum)) - min(np.array(result.bin_sum))
                    Result = result
                    print(f'results stored max corridor = {corridor_span} from {pair[0]} to {pair[1]}')
                    break
                        
        
    Result.to_csv("Ergebnis.csv")
    print(f' Untere Bin-Grenze= {min(np.array(Result.bin_sum))}')
    print(f' Obere Bin-Grenze= {max(np.array(Result.bin_sum))}')
    print(f' Maximale Varianz= {max(np.array(Result.bin_dif))}')
    
    return Result, corridor_span

In [92]:
def iterate_steps(data, tolerance, steps_list=[5, 10, 20, 50, 100], span=1):
    span = span
    for step in steps_list:
        results, span = iterate_narrowing_4(data, tolerance, step, span)
        print("#####", span, "#####")
    return "Success!"
    

In [ ]:
iterate_steps(data, tolerance)

1/9 - Testing: (1.06, 1.102)
2/9 - Testing: (1.06, 1.145)
3/9 - Testing: (1.06, 1.188)
4/9 - Testing: (1.102, 1.145)
5/9 - Testing: (1.102, 1.188)
6/9 - Testing: (1.102, 1.23)
results stored max corridor = 0.12450000000000006 from 1.102 to 1.23
7/9 - Testing: (1.145, 1.188)
8/9 - Testing: (1.145, 1.23)
9/9 - Testing: (1.188, 1.23)
 Untere Bin-Grenze= 1.1021
 Obere Bin-Grenze= 1.2266000000000001
 Maximale Varianz= 0.010599999999999998
##### 0.12450000000000006 #####
1/39 - Testing: (1.06, 1.079)
2/39 - Testing: (1.06, 1.098)
3/39 - Testing: (1.06, 1.117)
4/39 - Testing: (1.06, 1.136)
5/39 - Testing: (1.06, 1.154)
6/39 - Testing: (1.06, 1.173)
7/39 - Testing: (1.079, 1.098)
8/39 - Testing: (1.079, 1.117)
9/39 - Testing: (1.079, 1.136)
10/39 - Testing: (1.079, 1.154)
11/39 - Testing: (1.079, 1.173)
12/39 - Testing: (1.079, 1.192)
results stored max corridor = 0.11149999999999993 from 1.079 to 1.192
13/39 - Testing: (1.098, 1.117)
14/39 - Testing: (1.098, 1.136)
15/39 - Testing: (1.098, 1.

40/925 - Testing: (1.063, 1.116)
41/925 - Testing: (1.063, 1.119)
42/925 - Testing: (1.063, 1.122)
43/925 - Testing: (1.063, 1.126)
44/925 - Testing: (1.063, 1.129)
45/925 - Testing: (1.063, 1.133)
46/925 - Testing: (1.063, 1.136)
47/925 - Testing: (1.063, 1.14)
48/925 - Testing: (1.063, 1.143)
49/925 - Testing: (1.063, 1.147)
50/925 - Testing: (1.063, 1.15)
51/925 - Testing: (1.067, 1.07)
52/925 - Testing: (1.067, 1.074)
53/925 - Testing: (1.067, 1.077)
54/925 - Testing: (1.067, 1.081)
55/925 - Testing: (1.067, 1.084)
56/925 - Testing: (1.067, 1.088)
57/925 - Testing: (1.067, 1.091)
58/925 - Testing: (1.067, 1.095)
59/925 - Testing: (1.067, 1.098)
60/925 - Testing: (1.067, 1.102)
61/925 - Testing: (1.067, 1.105)
62/925 - Testing: (1.067, 1.109)
63/925 - Testing: (1.067, 1.112)
64/925 - Testing: (1.067, 1.116)
65/925 - Testing: (1.067, 1.119)
66/925 - Testing: (1.067, 1.122)
67/925 - Testing: (1.067, 1.126)
68/925 - Testing: (1.067, 1.129)
69/925 - Testing: (1.067, 1.133)
70/925 - Test

285/925 - Testing: (1.098, 1.133)
286/925 - Testing: (1.098, 1.136)
287/925 - Testing: (1.098, 1.14)
288/925 - Testing: (1.098, 1.143)
289/925 - Testing: (1.098, 1.147)
290/925 - Testing: (1.098, 1.15)
291/925 - Testing: (1.098, 1.154)
292/925 - Testing: (1.098, 1.157)
293/925 - Testing: (1.098, 1.161)
294/925 - Testing: (1.098, 1.164)
295/925 - Testing: (1.098, 1.168)
296/925 - Testing: (1.098, 1.171)
297/925 - Testing: (1.098, 1.174)
298/925 - Testing: (1.098, 1.178)
299/925 - Testing: (1.098, 1.181)
300/925 - Testing: (1.098, 1.185)
301/925 - Testing: (1.102, 1.105)
302/925 - Testing: (1.102, 1.109)
303/925 - Testing: (1.102, 1.112)
304/925 - Testing: (1.102, 1.116)
305/925 - Testing: (1.102, 1.119)
306/925 - Testing: (1.102, 1.122)
307/925 - Testing: (1.102, 1.126)
308/925 - Testing: (1.102, 1.129)
309/925 - Testing: (1.102, 1.133)
310/925 - Testing: (1.102, 1.136)
311/925 - Testing: (1.102, 1.14)
312/925 - Testing: (1.102, 1.143)
313/925 - Testing: (1.102, 1.147)
314/925 - Testing

523/925 - Testing: (1.129, 1.209)
524/925 - Testing: (1.129, 1.213)
525/925 - Testing: (1.129, 1.216)
526/925 - Testing: (1.133, 1.136)
527/925 - Testing: (1.133, 1.14)
528/925 - Testing: (1.133, 1.143)
529/925 - Testing: (1.133, 1.147)
530/925 - Testing: (1.133, 1.15)
531/925 - Testing: (1.133, 1.154)
532/925 - Testing: (1.133, 1.157)
533/925 - Testing: (1.133, 1.161)
534/925 - Testing: (1.133, 1.164)
535/925 - Testing: (1.133, 1.168)
536/925 - Testing: (1.133, 1.171)
537/925 - Testing: (1.133, 1.174)
538/925 - Testing: (1.133, 1.178)
539/925 - Testing: (1.133, 1.181)
540/925 - Testing: (1.133, 1.185)
541/925 - Testing: (1.133, 1.188)
542/925 - Testing: (1.133, 1.192)
543/925 - Testing: (1.133, 1.195)
544/925 - Testing: (1.133, 1.199)
545/925 - Testing: (1.133, 1.202)
546/925 - Testing: (1.133, 1.206)
547/925 - Testing: (1.133, 1.209)
548/925 - Testing: (1.133, 1.213)
549/925 - Testing: (1.133, 1.216)
550/925 - Testing: (1.133, 1.22)
551/925 - Testing: (1.136, 1.14)
552/925 - Testing:

767/925 - Testing: (1.168, 1.213)
768/925 - Testing: (1.168, 1.216)
769/925 - Testing: (1.168, 1.22)
770/925 - Testing: (1.168, 1.223)
771/925 - Testing: (1.168, 1.227)
772/925 - Testing: (1.168, 1.23)
773/925 - Testing: (1.171, 1.174)
774/925 - Testing: (1.171, 1.178)
775/925 - Testing: (1.171, 1.181)
776/925 - Testing: (1.171, 1.185)
777/925 - Testing: (1.171, 1.188)
778/925 - Testing: (1.171, 1.192)
779/925 - Testing: (1.171, 1.195)
780/925 - Testing: (1.171, 1.199)
781/925 - Testing: (1.171, 1.202)
782/925 - Testing: (1.171, 1.206)
783/925 - Testing: (1.171, 1.209)
784/925 - Testing: (1.171, 1.213)
785/925 - Testing: (1.171, 1.216)
786/925 - Testing: (1.171, 1.22)
787/925 - Testing: (1.171, 1.223)
788/925 - Testing: (1.171, 1.227)
789/925 - Testing: (1.171, 1.23)
790/925 - Testing: (1.174, 1.178)
791/925 - Testing: (1.174, 1.181)
792/925 - Testing: (1.174, 1.185)
793/925 - Testing: (1.174, 1.188)
794/925 - Testing: (1.174, 1.192)
795/925 - Testing: (1.174, 1.195)
796/925 - Testing:

83/3602 - Testing: (1.062, 1.122)
84/3602 - Testing: (1.062, 1.124)
85/3602 - Testing: (1.062, 1.125)
86/3602 - Testing: (1.062, 1.127)
87/3602 - Testing: (1.062, 1.129)
88/3602 - Testing: (1.062, 1.13)
89/3602 - Testing: (1.062, 1.132)
90/3602 - Testing: (1.062, 1.134)
91/3602 - Testing: (1.062, 1.136)
92/3602 - Testing: (1.062, 1.137)
93/3602 - Testing: (1.062, 1.139)
94/3602 - Testing: (1.062, 1.141)
95/3602 - Testing: (1.062, 1.142)
96/3602 - Testing: (1.062, 1.144)
97/3602 - Testing: (1.063, 1.065)
98/3602 - Testing: (1.063, 1.067)
99/3602 - Testing: (1.063, 1.069)
100/3602 - Testing: (1.063, 1.07)
101/3602 - Testing: (1.063, 1.072)
102/3602 - Testing: (1.063, 1.074)
103/3602 - Testing: (1.063, 1.075)
104/3602 - Testing: (1.063, 1.077)
105/3602 - Testing: (1.063, 1.079)
106/3602 - Testing: (1.063, 1.081)
107/3602 - Testing: (1.063, 1.082)
108/3602 - Testing: (1.063, 1.084)
109/3602 - Testing: (1.063, 1.086)
110/3602 - Testing: (1.063, 1.087)
111/3602 - Testing: (1.063, 1.089)
112/

320/3602 - Testing: (1.07, 1.129)
321/3602 - Testing: (1.07, 1.13)
322/3602 - Testing: (1.07, 1.132)
323/3602 - Testing: (1.07, 1.134)
324/3602 - Testing: (1.07, 1.136)
325/3602 - Testing: (1.07, 1.137)
326/3602 - Testing: (1.07, 1.139)
327/3602 - Testing: (1.07, 1.141)
328/3602 - Testing: (1.07, 1.142)
329/3602 - Testing: (1.07, 1.144)
330/3602 - Testing: (1.07, 1.146)
331/3602 - Testing: (1.07, 1.148)
332/3602 - Testing: (1.07, 1.149)
333/3602 - Testing: (1.07, 1.151)
334/3602 - Testing: (1.072, 1.074)
335/3602 - Testing: (1.072, 1.075)
336/3602 - Testing: (1.072, 1.077)
337/3602 - Testing: (1.072, 1.079)
338/3602 - Testing: (1.072, 1.081)
339/3602 - Testing: (1.072, 1.082)
340/3602 - Testing: (1.072, 1.084)
341/3602 - Testing: (1.072, 1.086)
342/3602 - Testing: (1.072, 1.087)
343/3602 - Testing: (1.072, 1.089)
344/3602 - Testing: (1.072, 1.091)
345/3602 - Testing: (1.072, 1.093)
346/3602 - Testing: (1.072, 1.094)
347/3602 - Testing: (1.072, 1.096)
348/3602 - Testing: (1.072, 1.098)


555/3602 - Testing: (1.079, 1.134)
556/3602 - Testing: (1.079, 1.136)
557/3602 - Testing: (1.079, 1.137)
558/3602 - Testing: (1.079, 1.139)
559/3602 - Testing: (1.079, 1.141)
560/3602 - Testing: (1.079, 1.142)
561/3602 - Testing: (1.079, 1.144)
562/3602 - Testing: (1.079, 1.146)
563/3602 - Testing: (1.079, 1.148)
564/3602 - Testing: (1.079, 1.149)
565/3602 - Testing: (1.079, 1.151)
566/3602 - Testing: (1.079, 1.153)
567/3602 - Testing: (1.079, 1.154)
568/3602 - Testing: (1.079, 1.156)
569/3602 - Testing: (1.079, 1.158)
570/3602 - Testing: (1.079, 1.16)
571/3602 - Testing: (1.079, 1.161)
572/3602 - Testing: (1.081, 1.082)
573/3602 - Testing: (1.081, 1.084)
574/3602 - Testing: (1.081, 1.086)
575/3602 - Testing: (1.081, 1.087)
576/3602 - Testing: (1.081, 1.089)
577/3602 - Testing: (1.081, 1.091)
578/3602 - Testing: (1.081, 1.093)
579/3602 - Testing: (1.081, 1.094)
580/3602 - Testing: (1.081, 1.096)
581/3602 - Testing: (1.081, 1.098)
582/3602 - Testing: (1.081, 1.099)
583/3602 - Testing: (

790/3602 - Testing: (1.087, 1.136)
791/3602 - Testing: (1.087, 1.137)
792/3602 - Testing: (1.087, 1.139)
793/3602 - Testing: (1.087, 1.141)
794/3602 - Testing: (1.087, 1.142)
795/3602 - Testing: (1.087, 1.144)
796/3602 - Testing: (1.087, 1.146)
797/3602 - Testing: (1.087, 1.148)
798/3602 - Testing: (1.087, 1.149)
799/3602 - Testing: (1.087, 1.151)
800/3602 - Testing: (1.087, 1.153)
801/3602 - Testing: (1.087, 1.154)
802/3602 - Testing: (1.087, 1.156)
803/3602 - Testing: (1.087, 1.158)
804/3602 - Testing: (1.087, 1.16)
805/3602 - Testing: (1.087, 1.161)
806/3602 - Testing: (1.087, 1.163)
807/3602 - Testing: (1.087, 1.165)
808/3602 - Testing: (1.087, 1.166)
809/3602 - Testing: (1.087, 1.168)
810/3602 - Testing: (1.089, 1.091)
811/3602 - Testing: (1.089, 1.093)
812/3602 - Testing: (1.089, 1.094)
813/3602 - Testing: (1.089, 1.096)
814/3602 - Testing: (1.089, 1.098)
815/3602 - Testing: (1.089, 1.099)
816/3602 - Testing: (1.089, 1.101)
